In [ ]:
#import dependencies

import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [ ]:
#Loading the data to pands dataframe
newsData = pd.read_csv('/content/FakeNewsTrainData.csv')

#First 5 rows of the data
newsData.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
newsData.shape

(20800, 5)

In [ ]:
#Checking for missing values
newsData.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
#Replacing the missing values with null string
newsData = newsData.fillna('')

In [ ]:
newsData.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
#Merge the author name and new title
newsData['content'] = newsData['author'] + ' ' + newsData['title']

In [ ]:
newsData.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
#Separating feature and target
X = newsData.drop(columns = 'label', axis = 1)
y = newsData['label']
print(X)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

**Stemming**:

Stemming is the process of reducing a word to its root word

In [ ]:
portStem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmedContent = re.sub('^a-zA-z',' ', content)
  stemmedContent = stemmedContent.lower()
  stemmedContent = stemmedContent.split()
  stemmedContent = [portStem.stem(word) for word in stemmedContent if not word in stopwords.words('english')]
  stemmedContent = ' '.join(stemmedContent)
  return stemmedContent

In [ ]:
newsData['content'] = newsData['content'].apply(stemming)

In [ ]:
X = newsData['content'].values
y = newsData['label'].values

In [ ]:
print(X)
print(y)

['darrel lucu hou dem aide: didn’t even see comey’ letter jason chaffetz tweet'
 'daniel j. flynn flynn: hillari clinton, big woman campu - breitbart'
 'consortiumnews.com truth might get fire' ...
 'michael j. de la merc rachel abram macy’ said receiv takeov approach hudson’ bay - new york time'
 'alex ansari nato, russia hold parallel exerci balkan'
 'david swanson keep f-35 aliv']
[1 0 1 ... 0 1 1]


In [ ]:
print(y.shape)

(20800,)


In [ ]:
#Converting the textual data to feature vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 19011)	0.27314143529531215
  (0, 16394)	0.23674771642101414
  (0, 11019)	0.33382699032194835
  (0, 10698)	0.2682074458620382
  (0, 9647)	0.22755933991034313
  (0, 8790)	0.20559637829119523
  (0, 6402)	0.21421418089718264
  (0, 5235)	0.2746736934005099
  (0, 4979)	0.25127860420630327
  (0, 4747)	0.33042766693086867
  (0, 3940)	0.2266346204366854
  (0, 3396)	0.3375505278043567
  (0, 806)	0.3646301064279561
  (1, 20324)	0.3013383222428995
  (1, 8569)	0.1993537731475076
  (1, 7067)	0.7158047516725045
  (1, 4712)	0.2642822047738531
  (1, 3768)	0.19178470652769988
  (1, 3093)	0.3734578306865317
  (1, 2706)	0.15554021122842912
  (1, 2251)	0.2945961305493189
  (2, 18929)	0.41491113753784553
  (2, 11824)	0.49151393723208897
  (2, 7615)	0.34605253138342823
  (2, 6934)	0.39293503470255664
  :	:
  (20797, 15070)	0.2719830381078626
  (20797, 14834)	0.2471904296997442
  (20797, 12631)	0.08017194340632139
  (20797, 11790)	0.1741363346438638
  (20797, 11714)	0.2931078361878451
  (20797, 11126)	0

Splitting the dataset to training and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state =2)

Training the Model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
#Evalution
X_train_prediction = model.predict(X_train)
X_train_accuracy = accuracy_score(X_train_prediction, y_train)

In [ ]:
print('Accuracy Score of the Training Data :',X_train_accuracy)

Accuracy Score of the Training Data : 0.9868990384615385


In [ ]:
#Accuracy score on Test Data
X_test_prediction = model.predict(X_test)
X_test_accuracy = accuracy_score(X_test_prediction, y_test)

In [ ]:
print('Accuracy Score of the Test data: ', X_test_accuracy)

Accuracy Score of the Test data:  0.9774038461538461


Make Predictive System

In [ ]:
newNews = X_test[5]

prediction = model.predict(newNews)
print(prediction)

if (prediction[0] == 0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [ ]:
print(y_test[5])

1
